In [2]:
#Imports
import json
from json import load
from datetime import datetime
from dateutil.relativedelta import relativedelta 
import pandas as pd

 #BOKEH!!
from bokeh.io import output_file, output_notebook, show
from bokeh.models import (
    GMapPlot, GMapOptions, ColumnDataSource, Circle, LogColorMapper, BasicTicker, ColorBar,
    DataRange1d, PanTool, WheelZoomTool, BoxSelectTool, Range1d
    )
from bokeh.models.mappers import ColorMapper, LinearColorMapper
from bokeh.palettes import RdYlGn7
from bokeh.io import export_png

data = pd.read_json('amsterdam_sold2.json', orient='records')
pp = pd.read_csv('geopost.csv')
keys = pd.read_json('Keys.json', orient='records')

In [3]:
#Filling variable with api keys from Keys.json
gmaps_keys = keys
gmaps_keys = gmaps_keys['key']

gmaps_key_lat_lon = gmaps_keys[0]
gmaps_key_bokeh = gmaps_keys[1]

In [4]:
#Data correction for values that are below certain price
datacorrect = data
datacorrect = datacorrect.sort_values(by=['price'])
datacorrect2 = datacorrect[datacorrect.price > 5000]

#Check the data
#export_csv = datacheck.to_csv (r'C:\Users\tramp\Documents\Python_project_houseM\datacheck.csv', index = None, header=True) 

In [5]:
#Filling DF with the values needed
processed_data = pd.DataFrame()
processed_data['gem4'] = datacorrect2['price'] / datacorrect2['area']
processed_data['postcode'] = datacorrect2['postal_code']
processed_data['adres'] = datacorrect2['address']
processed_data['plaats_dt'] = datacorrect2['posting_date']
processed_data['verkoop_dt'] = datacorrect2['sale_date']
processed_data['oppervlakte'] = datacorrect2['area']
processed_data['bouwjaar'] = datacorrect2['year_built']
processed_data['prijs'] = datacorrect2['price'] 

processed_pp = pd.DataFrame()
processed_pp['adres'] = pp['address']
processed_pp['latitude'] = pp['LAT']
processed_pp['longitude'] = pp['LON']


# Sort, Merge and Combine so that Google Geolocation can handle it
processed_data["adres"] = processed_data["adres"] + " " + processed_data["postcode"]
processed_data_pp = pd.merge(processed_data, processed_pp, on='adres', how='inner')
processed_data_pp
processed_data_pp['plaats_dt'] =  pd.to_datetime(processed_data_pp['plaats_dt'], format='%d-%m-%Y')
processed_data_pp = processed_data_pp.sort_values(by=['plaats_dt'])

In [6]:
#Google Geolocation address [adres] to longitude [lon] and latitude [lat]

#geopost = pd.DataFrame()
#geoposttop = pd.DataFrame()
#geocode_result = pd.DataFrame()
#geo = pd.DataFrame()

#geo = data
#geocode_result = pd.DataFrame()


#geopost["address"] = geo["address"] + " " + geo["postal_code"]
# geopost = geopost.head() testing purpose
#geopost

In [7]:
#import googlemaps

#gmaps_key = googlemaps.Client(key = gmaps_key_lat_lon)

#geopost["LAT"] = None
#geopost["LON"] = None

#for i in range(0, len(geopost), 1):
#    geocode_result = gmaps_key.geocode(geopost.iat[i,0])
#    try:
#        lat = geocode_result[0]["geometry"]['location']["lat"]
#        lon = geocode_result[0]["geometry"]['location']["lng"]
#        geopost.iat[i, geopost.columns.get_loc("LAT")] = lat
#        geopost.iat[i, geopost.columns.get_loc("LON")] = lon 
#    except:
#        lat = None
#        lon = None

#geopost
#geopost.to_csv('geopost.csv')
        

In [13]:
#A loop so that an image can be made per unit of time. 
#Every image can be converted to a GIF format via an external application.


pngteller = 0
teller = 1
for _ in range(5):
    
    intervalmin = 1
    intervalmax = intervalmin+ teller



    datefrommin = processed_data_pp['plaats_dt'].min()
    #the unit of time can be defined here
    datefrommax = datefrommin+ relativedelta(months=intervalmax)

    processed_data_pp2 = processed_data_pp[processed_data_pp.plaats_dt > datefrommin]
    processed_data_pp2 = processed_data_pp[processed_data_pp.plaats_dt < datefrommax] 
    datestamp = str(datefrommax)
   


    map_options = GMapOptions(lat=52.370216, lng=4.895168, map_type="roadmap", zoom=11)
    plot = GMapPlot(
        x_range=Range1d(), y_range=Range1d(), map_options=map_options
    )
    plot.title.text = "Huizen prijzen per m4 op "+datestamp

    # For GMaps to function, Google requires you obtain and enable an API key:
    #
    #     https://developers.google.com/maps/documentation/javascript/get-api-key
    #
    # Replace the value below with your personal API key:
    plot.api_key = gmaps_key_bokeh
    
    source = ColumnDataSource(
        data=dict(
            lat=processed_data_pp2.latitude.tolist(),
            lon=processed_data_pp2.longitude.tolist(),
            color=processed_data_pp2.gem4.tolist()
        )
    )

    max_median_house_value = processed_data_pp.loc[processed_data_pp['gem4'].idxmax()]['gem4']
    min_median_house_value = processed_data_pp.loc[processed_data_pp['gem4'].idxmin()]['gem4']


    color_mapper = LogColorMapper(palette=RdYlGn7, low=min_median_house_value, high=max_median_house_value)


    circle = Circle(x="lon", y="lat", size=5, fill_color={'field': 'color', 'transform': color_mapper}, fill_alpha=0.75, line_color=None)
    plot.add_glyph(source, circle)

    color_bar = ColorBar(color_mapper=color_mapper, ticker=BasicTicker(),
                         label_standoff=12, border_line_color=None, location=(0,0))
    plot.add_layout(color_bar, 'right')

    plot.add_tools(PanTool(), WheelZoomTool(), BoxSelectTool())

    #output_file("gmap_plot.html")
    #output_notebook()
    #show(plot)
    
    #selium is needed for a PNG output    
    pngteller = int(pngteller)
    pngteller = pngteller + 1
    pngteller = str(pngteller)

    export_png(plot, filename="plot"+pngteller+".png")
    print (teller)
    teller = teller + 1

1
2
3
4
5
